## Langchain Expression Language（LCEL）Quick Start

LCEL is a declarative way to compose chains and agents in LangChain, introduced to simplify building and managing chains of components like LLMs, tools, retrievers, etc.
Think of it as a functional programming layer over LangChain where you build pipelines using a clean, composable syntax (like Lego blocks).

### Beneifts

1. Composable Pipelines
You can use | to chain components, making it intuitive and readable.
Supports branching, merging, mapping, etc.

2. Type Safety & Debuggability
More predictable inputs/outputs between steps.
Easier to log, debug, and trace flows.

3. Better for Production
All chains are Runnable, so they support:
    Streaming
    Tracing
    Batching
    Concurrency

4. Unified Design
Works across agents, chains, retrievers, prompts, tools, etc.
Lets you build complex logic without subclassing or imperative glue code


#### Limitations

1. Learning Curve
It's newer and less documented than traditional LangChain chains.
May be harder for beginners without functional programming background.

2. Less Flexibility in Some Cases
For highly dynamic or stateful workflows (e.g. agent memory loops), LCEL may feel restrictive.

3. Ecosystem Still Evolving
Some wrappers, tools, or libraries might not support LCEL yet.
Not all legacy chains or components are easily portable.


### Using LCEL to Build an LLMChain（Prompt + LLM)

#### Pipe Operator

Use LCEL's `|` operator to connect these different components into a single chain.

**In this chain, the user's input is passed to the prompt template, then the output of the prompt is passed to the model, and finally the model's output is passed to the output parser.**


```python
chain = prompt | model | output_parser
```



In [1]:
from dotenv import load_dotenv


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate


In [4]:
model = ChatOpenAI(model="gpt-4o-mini")

In [5]:
prompt = ChatPromptTemplate.from_template("Tell a joke about {topic}")

# Initialize the output parser to convert the model's output into a string
output_parser = StrOutputParser()

# Build a processing chain that first generates a complete input using a prompt template, then processes it with the model, and finally parses the output.
chain = prompt | model | output_parser

# Call the processing chain and pass in the topic "x" to generate a joke about x.
chain.invoke({"topic": "software engineers"})

'Why do software engineers prefer dark mode?\n\nBecause light attracts bugs!'

### Step by Step Breakdown without LCEL

In [6]:
from langchain import PromptTemplate

# Prompt  Not using LCEL
prompt_template = PromptTemplate.from_template(
    "Tell a joke about {topic}"
)

In [7]:

prompt_value = prompt_template.invoke({"topic": "software engineers"})
prompt_value

StringPromptValue(text='Tell a joke about software engineers')

In [8]:
prompt_value.to_messages()

[HumanMessage(content='Tell a joke about software engineers', additional_kwargs={}, response_metadata={})]

In [9]:
prompt_value.to_string()

'Tell a joke about software engineers'

In [10]:
# GPT-4o-mini model is a ChatModel, and its invoke method will return a BaseMessage.
message = model.invoke(prompt_value)

In [11]:
message

AIMessage(content='Why do software engineers prefer dark mode?\n\nBecause light attracts bugs!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 13, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_44added55e', 'finish_reason': 'stop', 'logprobs': None}, id='run-fdc06e0c-f8ce-461c-94d5-78b16fe16685-0', usage_metadata={'input_tokens': 13, 'output_tokens': 14, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [12]:
# The message is processed by StrOutputParser and converted into a standard string.
output_parser.invoke(message)

'Why do software engineers prefer dark mode?\n\nBecause light attracts bugs!'

In [19]:
# using an LLM model like gpt-3.5-turbo-instruct, the invoke method will return a string.

from langchain_openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
llm.invoke(prompt_value)

'\nWhy was the software engineer afraid of the debugger?\n\nBecause it kept finding all his bugs!'

## Invoke In Retrieval (RAG)


```python
chain = setup_and_retrieval | prompt | model | output_parser
```

First, set up a tretriever using a memory-based store that can fetch documents based on a query. This retriever is also a runnable component and can be linked with other components, can also running it along.

Steps:

1. First create a `RunnableParallel` object **setup_and_retrieval** with two entries。the first entry `context` will include the document results retrieved by the retriever. The second entry `question` will contain the original user question. To pass through the question, we use `RunnablePassthrough` to copy that entry,
2. The dictionary from the previous step is passed to the `Prompt` component. It receives user input along with the retrieved documents (i.e., the context), construct the prompt, and outputs a PromptValue.
3. `Model` component receives the generated prompt and passes it to the Opean AI `gpt-4o-mini` model for evaluation。The output generated by the model is a ChatMessage object.
4. Finally, `output_parser` component receives the ChatMessage and converts it into a Python string, which is then retruned by the invoke method.


In [20]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

# Create an in-memory vector store using DocArrayInMemorySearch
# Use OpenAIEmbeddings to generate embedding vectors for the texts
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings(),
)

# Convert the vector store into a retriever.
retriever = vectorstore.as_retriever()

# Prompt template
template = """Answer the question based on the following context:
{context}

question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

output_parser = StrOutputParser()

# Set up a parallel runner to handle context retrieval and question passing simultaneously
# Use RunnableParallel to prepare the expected input by combining the retrieved document entries and the original user question
# Perform document search using the retriever, and use RunnablePassthrough to pass through the user's question.
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)

chain = setup_and_retrieval | prompt | model | output_parser

chain.invoke("where does harrison work？")

#### Ignore UserWarning：

UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.

Cause：`The issue is with pydantic version, it's 2.0.0+ and not compatible with docarray.
Instead it should be pydantic==1.10.9`

Reference：https://github.com/langchain-ai/langchain/issues/15394

LangChain's document on Pydantic compatibility：https://python.langchain.com/v0.1/docs/guides/development/pydantic_compatibility/
